In [86]:
import os
import json
import pandas as pd
import traceback

In [87]:
from langchain.chat_models import ChatOpenAI

In [88]:
from dotenv import load_dotenv

load_dotenv() 

True

In [89]:
KEY=os.getenv("OPENAI_API_KEY")

In [90]:
print(KEY)

sk-proj-atcTdTP9Q2LYP9VsOUCMT3BlbkFJT4P6m8KfwPAuJYdBpf8a


In [91]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [92]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000211666D3040>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000211666D57C0>, temperature=0.5, openai_api_key='sk-proj-atcTdTP9Q2LYP9VsOUCMT3BlbkFJT4P6m8KfwPAuJYdBpf8a', openai_proxy='')

In [93]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [94]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "4":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
}

In [95]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [96]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [97]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [98]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [99]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [100]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [101]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [102]:
file_path=r"C:\Users\31ash\Desktop\mcq_generator\data.txt"

In [103]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [104]:
print(TEXT)

ChatGPT stands as a pinnacle of technical achievement in the realm of artificial intelligence, boasting an impressive array of capabilities honed through state-of-the-art machine learning techniques. At its core lies a sophisticated neural network architecture, meticulously trained on vast datasets encompassing diverse linguistic patterns and knowledge domains. Through the marvel of deep learning, ChatGPT exhibits a profound understanding of language semantics, syntax, and context, enabling it to comprehend and generate text with remarkable fidelity.

One of its defining features is its ability to engage in contextually relevant conversations, seamlessly tracking the flow of dialogue and adapting responses accordingly. Leveraging techniques such as attention mechanisms and transformer architectures, ChatGPT excels at capturing long-range dependencies within text, facilitating coherent and contextually grounded interactions.

Furthermore, ChatGPT demonstrates impressive language generat

In [105]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [116]:
NUMBER=5 
SUBJECT="TRIGONOMETRY"
TONE="hard"

In [117]:
#setting up Token usage tracking in langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:ChatGPT stands as a pinnacle of technical achievement in the realm of artificial intelligence, boasting an impressive array of capabilities honed through state-of-the-art machine learning techniques. At its core lies a sophisticated neural network architecture, meticulously trained on vast datasets encompassing diverse linguistic patterns and knowledge domains. Through the marvel of deep learning, ChatGPT exhibits a profound understanding of language semantics, syntax, and context, enabling it to comprehend and generate text with remarkable fidelity.

One of its defining features is its ability to engage in contextually relevant conversations, seamlessly tracking the flow of dialogue and adapting responses accordingly. Leveraging techniques such as attention mechanisms and transformer architectures, ChatGPT excels at capturing long-range dependencies within text, facilitating coh

In [118]:
print(f"total tokens:{cb.total_tokens}")
print(f"Prompt tokens:{cb.prompt_tokens}")
print(f"Completion tokens:{cb.completion_tokens}")
print(f"total cost:{cb.total_cost}")

total tokens:2191
Prompt tokens:1418
Completion tokens:773
total cost:0.003673


In [119]:
quiz=response.get("quiz")

In [120]:
type(quiz)

str

In [121]:
quiz=json.loads(quiz)

In [122]:
quiz

{'1': {'mcq': "What is the primary focus of ChatGPT's capabilities in the realm of artificial intelligence?",
  'options': {'a': 'Image recognition',
   'b': 'Language semantics and context',
   'c': 'Robotics',
   'd': 'Weather forecasting'},
  'correct': 'b'},
 '2': {'mcq': 'How does ChatGPT demonstrate proficiency in language generation?',
  'options': {'a': 'By performing complex mathematical calculations',
   'b': 'By generating fluent and contextually appropriate responses',
   'c': 'By composing music',
   'd': 'By analyzing chemical compounds'},
  'correct': 'b'},
 '3': {'mcq': 'What techniques does ChatGPT leverage to excel at capturing long-range dependencies within text?',
  'options': {'a': 'Attention mechanisms and transformer architectures',
   'b': 'Quantum computing',
   'c': 'Genetic algorithms',
   'd': 'Blockchain technology'},
  'correct': 'a'},
 '4': {'mcq': 'In what tasks does ChatGPT showcase proficiency beyond text generation?',
  'options': {'a': 'Image editing